In [41]:
par_storage = "testsynpriv2stor"
par_container = "test"
par_csv_file = 'testcsv.txt'

StatementMeta(test, 2, 53, Finished, Available)

In [42]:
import socket

addr = socket.gethostbyname(par_storage + '.dfs.core.windows.net')

print(addr)

StatementMeta(test, 2, 54, Finished, Available)

10.250.0.10

In [43]:
%%pyspark
# Proof that synapse workspace can load data from attached storage account using private IP
df = spark.read.load('abfss://' + par_container + '@' + par_storage + '.dfs.core.windows.net/' + par_csv_file, format='text')
display(df.limit(10))

StatementMeta(test, 2, 55, Finished, Available)

SynapseWidget(Synapse.DataFrame, d0638561-d193-4ae3-aa38-eb2f1b5feac2)

In [44]:
%%spark

val par_sqldw_server = "test-edlprod1-dbs.database.windows.net"
val par_sqldw_db_schema_table = "test-synapse-sqldw.dbo.AdultCensusIncome4"

// SQLDW outside Synapse analytics. In this example, SQLDW table data originates from csv in this folder: AdultCensusIncome_withheader.csv. Make sure that the Synapse Workspace MI is registered as external user in the external SQLDWimport com.microsoft.spark.sqlanalytics.utils.Constants
import org.apache.spark.sql.SqlAnalyticsConnector._

val df = spark.read.
option(Constants.SERVER, par_sqldw_server).
sqlanalytics(par_sqldw_db_schema_table)
df.createOrReplaceTempView("temp_table")

StatementMeta(test, 2, 56, Finished, Available)

par_sqldw_server: String = test-edlprod1-dbs.database.windows.net
par_sqldw_db_schema_table: String = test-synapse-sqldw.dbo.AdultCensusIncome4
import org.apache.spark.sql.SqlAnalyticsConnector._
df: org.apache.spark.sql.DataFrame = [age: int, workclass: string ... 13 more fields]


In [45]:
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

# To be able to complete this lab in under an hour, let's just work with 1,500k rows
data_all = spark.sql("SELECT * FROM temp_table").limit(15000000)
#sample.createOrReplaceTempView("AdultCensusIncome")

columns_new = [col.replace("-", "") for col in data_all.columns]
data_all = data_all.toDF(*columns_new)

StatementMeta(test, 2, 57, Finished, Available)

In [46]:
data_all.printSchema()

StatementMeta(test, 2, 58, Finished, Available)

root
 |-- age: integer (nullable = false)
 |-- workclass: string (nullable = false)
 |-- fnlwgt: integer (nullable = false)
 |-- education: string (nullable = false)
 |-- educationnum: integer (nullable = false)
 |-- maritalstatus: string (nullable = false)
 |-- occupation: string (nullable = false)
 |-- relationship: string (nullable = false)
 |-- race: string (nullable = false)
 |-- sex: string (nullable = false)
 |-- capitalgain: integer (nullable = false)
 |-- capitalloss: integer (nullable = false)
 |-- hoursperweek: integer (nullable = false)
 |-- nativecountry: string (nullable = false)
 |-- income: string (nullable = false)

In [47]:
# In case public access is disabled from the storage account attached to Synapse, Spark pool can't create the temp table and stack trace below is created
# In case public access is enabled in the storage data, temp table can be created on storage account and data can be loaded correctly
#
# It seems that org.apache.spark.sql.SqlAnalyticsConnector._ tries to acces to storage account using the public DNS rather than the private DNS

display(data_all.limit(10))

StatementMeta(test, 2, 59, Finished, Available)

Py4JJavaError: An error occurred while calling z:com.microsoft.spark.notebook.visualization.display.executeWithReturnVal.
: com.microsoft.spark.sqlanalytics.exception.SQLAnalyticsConnectorException: com.microsoft.sqlserver.jdbc.SQLServerException: CREATE EXTERNAL TABLE AS SELECT statement failed as the path name 'abfss://synapse@testsynpriv2stor.dfs.core.windows.net/synapse/workspaces/test-synpriv2-syn/sparkpools/test/sparkpoolinstances/e1594f0f-768c-45cb-b84d-7719fb0ec499/livysessions/2021/11/09/2/tempdata/SQLAnalyticsConnectorStaging/application_1636443688496_0002/aUUYkzHI5xA6181fe1f98cd4afb931814c9d6c6f84d.tbl' could not be used for export. Please ensure that the specified path is a directory which exists or can be created, and that files can be created in that directory.
	at com.microsoft.spark.sqlanalytics.read.SQLAnalyticsReader$PlanInputPartitionsUtilities$.extractDataAndGetLocation(SQLAnalyticsReader.scala:256)
	at com.microsoft.spark.sqlanalytics.read.SQLAnalyticsReader.planBa